# BerryBox Berry Segmentation

## Run inferencing using a trained YOLOv8 model

This notebook is meant to be run on a Windows PC using a CPU

0. Install depedencies

In [1]:
# Install dependencies
%pip install -r requirements.txt

# Open ultralytics
import ultralytics
ultralytics.checks()

# Set project directory and change directory
import os
proj_dir = os.getcwd()

Ultralytics YOLOv8.0.186  Python-3.9.18 torch-2.2.1+cpu CPU (Intel Core(TM) i7-10810U 1.10GHz)
Setup complete  (12 CPUs, 31.6 GB RAM, 483.3/951.3 GB disk)


1. Import a YOLO model and export it using OpenVINO

In [2]:
# Set the path to the model
# model_path = os.path.join(proj_dir, "models/berrybox_best_20240316.pt")
model_path = os.path.join(proj_dir, "models/berrybox_inst_seg_nano_best_20240318.pt")

# Attempt to find the openvino version of the model;
# If it does not exist, export the model
if not os.path.exists(model_path.replace(".pt", "_openvino_model")):

    # Load the model with YOLO
    from ultralytics import YOLO
    model = YOLO(model_path)

    # Export the model using openVINO
    # model.export(format = "openvino", imgsz = 2048, half = True)
    model.export(format = "openvino", imgsz = (1344, 2016), half = True)

Ultralytics YOLOv8.0.186  Python-3.9.18 torch-2.2.1+cpu CPU (Intel Core(TM) i7-10810U 1.10GHz)
YOLOv8n-seg summary (fused): 195 layers, 3258844 parameters, 0 gradients

PyTorch: starting from 'c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging\models\berrybox_inst_seg_nano_best_20240318.pt' with input shape (1, 3, 1344, 2016) BCHW and output shape(s) ((1, 40, 55566), (1, 32, 336, 504)) (7.0 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success  26.2s, saved as 'c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging\models\berrybox_inst_seg_nano_best_20240318.onnx' (13.5 MB)

OpenVINO: starting export with openvino 2024.0.0-14509-34caeefd078-releases/2024/0...
OpenVINO: export success  249.3s, saved as 'c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging\models\berrybox_inst_seg_nano_best_20240318_openvino_model\' (6.9

2. Change Working Directory (Choose)

In [3]:
# Set project directory
import os
proj_dir = os.getcwd()

%ls

 Volume in drive C is OS
 Volume Serial Number is 5E84-0237

 Directory of c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging

03/17/2024  07:01 PM    <DIR>          .
03/17/2024  07:01 PM    <DIR>          ..
03/17/2024  03:42 PM    <DIR>          .ipynb_checkpoints
03/17/2024  04:58 PM    <DIR>          __pycache__
12/29/2022  11:40 AM             2,696 03_BerryBox_2021ImageAnalysis_DataCleanup.R
03/17/2024  10:55 PM            61,433 berrybox_yolov8_inference.ipynb
07/24/2023  04:44 PM    <DIR>          BerrySizeValidationImages
01/01/1980  12:00 AM            13,618 functions.py
03/17/2024  06:19 PM    <DIR>          images
11/25/2022  01:01 PM    <DIR>          lightbox
03/18/2024  08:58 PM    <DIR>          models
03/17/2024  05:56 PM    <DIR>          raw_images
11/22/2022  10:27 AM               464 REAMDE.txt
03/17/2024  04:41 PM                79 requirements.txt
03/17/2024  05:33 PM    <DIR>          test_images
01/04/2023  

3. Import dependecies, set parameters
Note: Make sure functions.py is in the current working directory

In [5]:
from functions import * # load all functions
from ultralytics import YOLO
import os
import torch
import gc
import shutil

gc.collect()   # collect garbage

device = '0' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')


"""
------------------------------------------------------------------------------------
Set Directories
------------------------------------------------------------------------------------
"""
model_dir = 'models/berrybox_best_20240316_openvino_model' # path to the model
image_dir = 'images' # path to the image folder
save_dir = 'output' # path to save the results

shutil.rmtree(save_dir, ignore_errors=True)

"""
------------------------------------------------------------------------------------
Set Model Parameters (you can change these parameters to suit your needs)
------------------------------------------------------------------------------------
"""
model_params = {
    'project': save_dir, # project name
    'name': '2021_BerryBox', # run name
    'save': False, # save image results
    'show_labels': True,   # hide labels
    'show_conf': True, # hide confidences
    'save_crop': False, # save cropped prediction boxes
    'line_width': 3, # bounding box line width
    'conf': 0.70, # confidence threshold
    'iou': 0.75, # NMS IoU threshold
    # 'imgsz': 2048,
    'imgsz': (1344, 2016),
    # 'imgsz': (640, 960),
    # 'exist_ok': True, # if True, it overwrites current 'name' saving folder
    'half': True, # use FP16 half-precision inference True/False
    'cache': False, # use cache images for faster inference
    'retina_masks': False, #use high resolution seg mask
    'device': device # cuda device, i.e. 0 or 0,1,2,3 or cpu
}


Using device: cpu


4. Run Inference

In [6]:
print('1. Running inference...')
model = YOLO(model_dir, task = "segment")
results = model.predict(source = image_dir, **model_params)

1. Running inference...


Loading models\berrybox_best_20240316_openvino_model for OpenVINO inference...

WARNING  NMS time limit 0.550s exceeded
image 1/7 c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging\test_images\DSC_0287.JPG: 1344x2016 1 ColorCard, 33 berrys, 1 info, 40642.3ms
image 2/7 c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging\test_images\DSC_0288.JPG: 1344x2016 1 ColorCard, 34 berrys, 1 info, 39485.2ms


5. Unscramble the predicted results/ Assemble features from detections, Save results

In [ ]:
print('2. Extracting features from results...')
DF = pd.DataFrame()
for i, result in enumerate(results):

    result = result.cpu()
    img_name = os.path.basename(result.path)

    result, patch_size = color_correction(result)
    QR_info = read_QR_code(result)
    df = get_all_features_parallel(result, name= 'berry')
    w,_ = df.shape
    img_name = [img_name]*w
    QR_info = [QR_info]*w
    patch_size = [np.mean(patch_size)]*w
    indeces = list(range(w))

    df_fore = pd.DataFrame({'Image_name': img_name,
                            'ID': indeces,
                            'QR_info': QR_info,
                            'Patch_size': patch_size})

    df = pd.concat([df_fore, df], axis=1)
    DF = pd.concat([DF, df], axis=0, ignore_index=True)

    img_save_folder = os.path.join(save_dir, 'Predictions')
    if not os.path.exists(img_save_folder):
        os.makedirs(img_save_folder)

    save_ROI_parallel(result,
                get_ids(result, 'berry'),
                os.path.join(img_save_folder, img_name[0]))

    print(f"\nImage {i+1} of {len(results)} processed." )

DF.to_csv(os.path.join(save_dir, 'Features.csv'), index=False)

print('Done.')

gc.collect()

In [19]:
# # Rotate all images in a directory
# from PIL import Image
# images = os.listdir("raw_images/")
# for img_file in images:
#     img = Image.open("raw_images/" + img_file)
#     sz = img.size
#     if (sz != (5568, 3712)):
#         img = img.resize((5568, 3712))

#     img.save("images/" + img_file)

